In [ ]:
import sys
import os
import requests

#Load the env details
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Get environment variables
apim_url = os.getenv("APIM_URL")
api_version = os.getenv("APIM_API_VERSION")
subscription_key = os.getenv("APIM_SUBSCRIPTION_KEY")

embedding_deployment_name = os.getenv("APIM_EMBEDDING_DEPLOYMENT_NAME")
chat_deployment_name = os.getenv("APIM_CHAT_DEPLOYMENT_NAME")

# Construct the URL and headers
embedding_url = f'{apim_url}/deployments/{embedding_deployment_name}/embeddings?api-version={api_version}'
chat_url = f'{apim_url}/deployments/{chat_deployment_name}/chat/completions?api-version={api_version}'

headers = {
    "Content-Type": "application/json",
    "Ocp-Apim-Subscription-Key": subscription_key
}

In [ ]:
# Test Connection: EMBEDDING

# Sample input text
input_text = "Hello, World"

# Make the request
response = requests.post(embedding_url, headers=headers, json={"input": input_text})

# Print the response
print(f"Response status code: {response.status_code}")

# Check if the request was successful
if response.status_code == 200:
    embedding = response.json().get("data", [])[0].get("embedding", [])
    print(f"Embedding: {embedding}")
else:
    print("Failed to get embedding")

In [ ]:
# Test Failover: EMBEDDING

input_text = "Hello, World!"

total_calls = 30
failure = 0
failure_reasons = {}

for i in range(total_calls):
    response = requests.post(embedding_url, headers=headers, json={"input": input_text})
    if response.status_code == 200:
        embedding = response.json().get("data", [])[0].get("embedding", [])[0]
        print(f"[{i+1}/{total_calls}] Status {response.status_code}: {embedding} ...")
    else:
        failure += 1
        failure_reasons.update({response.status_code: i})   # Keep track of different failure http code returned
        print(f"[{i+1}/{total_calls}] Status {response.status_code}: Failed to get embedding")
        

print(f"\n{(total_calls-failure)} succeeded out of {total_calls} calls")
if failure > 0:
    print(f"\n{failure} failure with response {', '.join(map(str, failure_reasons.keys()))}")

In [ ]:
# Test Connection: CHAT

# Sample input text
input_text = {
        "messages": [
            {
                "role": "user",
                "content": "Hey. What's up?"
            }
        ],
    }

# Make the request
response = requests.post(chat_url, headers=headers, json=input_text)

# Print the response
print(f"Response status code: {response.status_code}")

# Check if the request was successful
if response.status_code == 200:
    chat = response.json()['choices'][0]['message']['content']
    print(f"Chat: {chat}")
else:
    print("Failed to get chat")

In [ ]:
# Test Failover: CHAT

input_text = {
        "messages": [
            {
                "role": "user",
                "content": "Hey. What's up?"
            }
        ],
    }

total_calls = 30
failure = 0
failure_reasons = {}

for i in range(total_calls):
    response = requests.post(chat_url, headers=headers, json=input_text)
    if response.status_code == 200:
        chat = response.json()['choices'][0]['message']['content'][:6]
        print(f"[{i+1}/{total_calls}] Status {response.status_code}: {chat} ...")
    else:
        failure += 1
        failure_reasons.update({response.status_code: i})   # Keep track of different failure http code returned
        print(f"[{i+1}/{total_calls}] Status {response.status_code}: Failed to get chat")

print(f"\n{(total_calls-failure)} succeeded out of {total_calls} calls")
if failure > 0:
    print(f"\n{failure} failure with response {', '.join(map(str, failure_reasons.keys()))}")